### 1. 介绍

- **关键记忆**：
  1. 转换向量$w$和$x$的关系是：$x_i=A_iw$
  2. $\bf{A}$ 是一个张量，记得新增了克罗内克积$\otimes$
  3. $S_{ij}(A_i^\mathrm{T}A_i-A_i^\mathrm{T}A_j)$
  4. 联想LPP，记得最后解的形式：$\mathbf{A}^\mathrm{T}(L\otimes I_m)\mathbf{A}w=\lambda\mathbf{A}^\mathrm{T}(D\otimes I_m)\mathbf{A}w$

- **提出背景**：
    - LPP被提出用于降维，它可以检测数据的内在流形结构并保留局部信息。
    - 对于矩阵数据（如图像），通常会将其向量化，以便LPP算法可以找到内在流形结构。
    - 然而，由于矩阵数据的维度通常非常高，LPP因矩阵的奇异性而无法实现（小样本问题）

- **2D-LPP 方法**：用于图像识别，直接基于二维图像矩阵。
- **优势**：
  - 保留了图像空间的欧几里得结构，嵌入方式保留了局部信息并检测出内在的图像流形结构。
  - 识别率高，计算复杂度低


- **流程**：

  1. 目标函数：$$\min \sum_{i < j} S_{ij} \|x_i - x_j\|^2$$

  2. 假设 $w$ 是一个变换向量 $x_i = A_i w$，然后令$\mathbf{A}^\mathrm{T} = [A_1^\mathrm{T}, A_2^\mathrm{T}, \ldots, A_N^\mathrm{T}]$，$D$ 是一个对角矩阵，对角线元素为 $D_{ii} = \sum_j S_{ij}$。$L = D - S$ 是拉普拉斯矩阵。则有：
  $$
  \begin{aligned}
  & \sum_{i < j} S_{ij} \|x_i - x_j\|^2 = \sum_{i < j} S_{ij} \|(A_i - A_j) w\|^2 \\
  & = w^\mathrm{T} \left(\sum_{ij} S_{ij}(A_i^\mathrm{T}A_i - A_i^\mathrm{T}A_j)\right) w \\
  & = w^\mathrm{T} \left(\sum_{i,j} A_i^\mathrm{T} S_{ij} I_m A_i - \sum_{i,j} A_i^\mathrm{T} S_{ij} I_m A_j\right) w \\
  & = w^\mathrm{T} \mathbf{A}^\mathrm{T} (L \otimes I_m) \mathbf{A} w,
  \end{aligned}
  $$

  1. 施加如下约束：$$\sum_i D_{ii} X_i^\mathrm{T} X_i = 1 \Rightarrow w^\mathrm{T} \left(\sum_i D_{ii} A_i^\mathrm{T} A_i\right) w = w^\mathrm{T} \mathbf{A}^\mathrm{T} (D \otimes I_m) \mathbf{A} w = 1$$
  2. 最小化问题变为：$$\underset{w^\mathrm{T} \mathbf{A}^\mathrm{T} (D \otimes I_m) \mathbf{A} w = 1}{\operatorname*{\arg\min}} w^\mathrm{T} \mathbf{A}^\mathrm{T} (L \otimes I_m) \mathbf{A} w$$故最小化目标函数的变换向量 $w$ 由广义特征值问题的最小特征向量解给出：$$\mathbf{A}^\mathrm{T} (L \otimes I_m) \mathbf{A} w = \lambda \mathbf{A}^\mathrm{T} (D \otimes I_m) \mathbf{A} w. $$

### 2. 流程

#### 2.1. 二维局部保持投影

现在我们考虑一组 $N$ 个样本图像 $A_1, A_2, \ldots, A_N$，这些图像来自于一个 $(m \times n)$ 维的图像空间。由于维度较高，直接应用传统算法进行识别是困难的，因此降维尤为重要。我们设计了一种线性变换，将原始的 $(m \times n)$ 维图像空间映射到 $m$ 维特征空间。设 $w$ 为一个 $n$ 维的单位列向量。这里提出的方法是通过以下变换将每个图像 $A_i$（一个 $(m \times n)$ 矩阵）投影到 $w$ 上：


$$
x_i = A_i w, \quad i = 1, 2, \ldots, N.
$$

这样，我们为每个图像 $A_i$ 得到了一个 $m$ 维的投影特征 $x_i$。为了达到最佳的识别效果，选择一个好的投影向量 $w$ 是非常重要的。

#### 2.2. 算法

二维局部保持投影（2D-LPP）是LPP的二维扩展。我们的算法结构可以正式表述如下：

1. **构建最近邻图：** 设 $G$ 表示一个包含 $N$ 个节点的图，第 $i$ 个节点对应图像 $A_i$。如果 $A_i$ 和 $A_j$ “接近”，我们在节点 $i$ 和 $j$ 之间添加一条边。测量“接近”的方法有几种：
   - **k 最近邻：** 如果 $i$ 是 $j$ 的 $k$ 个最近邻之一，或 $j$ 是 $i$ 的 $k$ 个最近邻之一，则节点 $i$ 和 $j$ 之间有一条边。
   - **$\varepsilon$ 邻域：** 如果 $\|A_i - A_j\| < \varepsilon$，则节点 $i$ 和 $j$ 之间有一条边，其中两个矩阵之间的距离 $\|\cdot\|$ 是它们在 $R^{mn}$ 中向量化表示的欧几里得距离。
   注意：在这里，我们可以添加训练样本的标签信息（如果有），以提高判别性能，这可以通过限制每个图像的 $k$ 最近邻来自同一类来实现。在本文的实验中，我们在这里添加了训练图像的标签信息。

2. **选择权重：** 如果节点 $i$ 和 $j$ 之间有一条边，则在其上放置一个相似度权重 $S_{ij}$，否则令 $S_{ij} = 0$。然后我们得到一个稀疏对称的 $(N \times N)$ 相似度矩阵 $S$。相似度权重 $S_{ij}$ 可以是：
   - **简单的：** 当且仅当节点 $i$ 和 $j$ 由一条边连接时，$S_{ij} = 1$。
   - **热核：** 如果节点 $i$ 和 $j$ 由一条边连接，则 $S_{ij} = \exp\{-\|A_i - A_j\|^2 / t\}$，其中 $t$ 是一个合适的常数。在本文中，我们选择 $t$ 为成对邻近图像的最大距离的常数倍。

3. **特征映射：** 计算广义特征值问题的特征向量和特征值：
   $$
   \mathbf{A}^\mathrm{T}(L \otimes I_m)\mathbf{A}w = \lambda \mathbf{A}^\mathrm{T}(D \otimes I_m)\mathbf{A}w,
   $$
   其中 $D$ 是一个对角矩阵，$D_{ii} = \sum_j S_{ij}$。$L = D - S$ 是拉普拉斯矩阵。$\mathbf{A}$ 是一个 $(mN \times n)$ 矩阵，通过排列所有图像矩阵得到：
   $$
   \mathbf{A} = [A_1^\mathrm{T}, A_2^\mathrm{T}, \ldots, A_N^\mathrm{T}]^\mathrm{T}.
   $$
   算子 $\otimes$ 是矩阵的克罗内克积。$I_m$ 是阶数为 $m$ 的单位矩阵。
   令 $w_1, w_2, \ldots, w_d$ 为方程 (11) 的前 $d$ 个单位正交解向量，对应于最小的 $d$ 个广义特征值，按其大小排列，$0 \leqslant \lambda_1 \leqslant \lambda_2 \leqslant \cdots \leqslant \lambda_d$。这些特征值是非负的，因为矩阵 $\mathbf{A}^\mathrm{T}(L \otimes I_m)\mathbf{A}$ 和 $\mathbf{A}^\mathrm{T}(D \otimes I_m)\mathbf{A}$ 都是对称且半正定的。嵌入如下：
   $$
   A_i \to X_i = A_i W, \quad W = (w_1, w_2, \ldots, w_d),
   $$
   $$
   i = 1, 2, \ldots, N,
   $$
   其中 $X_i$ 是 $A_i$ 的 $(m \times d)$ 特征矩阵，$W$ 是一个 $(n \times d)$ 变换矩阵。

经过2D-LPP训练后，每个图像的特征矩阵和一个变换矩阵被获得。然后可以使用最近邻分类器进行分类。这里两个特征矩阵之间的距离定义为 $X_i = (x_1^{(i)}, x_2^{(i)}, \ldots, x_d^{(i)})$ 和 $X_j = (x_1^{(j)}, x_2^{(j)}, \ldots, x_d^{(j)})$ 之间的距离为 $l(X_i, X_j) = \sum_{k=1}^d \|x_k^{(i)} - x_k^{(j)}\|$。假设 $X_1, X_2, \ldots, X_N$ 是训练图像 $A_1, A_2, \ldots, A_N$ 的特征矩阵，每个图像有一个类别标签 $C_k$。给定一个测试图像 $A_0$，其特征矩阵 $X_0$ 可以通过 $X_0 = A_0 W$ 获得。如果 $d(X_0, X_j) = \min_{i} d(X_0, X_i)$ 并且 $A_j \in \mathcal{C}_k$，则我们判断 $A_0 \in \mathcal{C}_k$。

#### 2.3. 理论依据

给定一组训练图像 $A_i, i = 1, 2, \ldots, N$，我们构建了一个相似度矩阵 $S$。考虑将 $(m \times n)$ 维图像空间中的一个点映射到 $m$ 维欧几里得空间中的一个点的问题，使得连接的点尽可能靠近，并保留数据的内在几何结构和局部结构。设 $x_i$ 为 $A_i, i = 1, 2, \ldots, N$ 的映射。选择这个映射的合理标准是最小化以下目标函数：


$$
\min \sum_{i < j} S_{ij} \|x_i - x_j\|^2.
$$

当邻近点 $A_i$ 和 $A_j$ 映射得很远时，权重 $S_{ij}$ 会带来很大的惩罚。因此，最小化目标函数是为了确保如果 $A_i$ 和 $A_j$ “接近”，则 $x_i$ 和 $x_j$ 也接近。假设 $w$ 是一个变换向量 $x_i = A_i w$。通过简单的代数运算，我们可以看到：

$$
\begin{aligned}
& \sum_{i < j} S_{ij} \|x_i - x_j\|^2 = \sum_{i < j} S_{ij} \|(A_i - A_j) w\|^2 \\
& = w^\mathrm{T} \left(\sum_{ij} S_{ij}(A_i^\mathrm{T}A_i - A_i^\mathrm{T}A_j)\right) w \\
& = w^\mathrm{T} \left(\sum_{i,j} A_i^\mathrm{T} S_{ij} I_m A_i - \sum_{i,j} A_i^\mathrm{T} S_{ij} I_m A_j\right) w \\
& = w^\mathrm{T} \mathbf{A}^\mathrm{T} (L \otimes I_m) \mathbf{A} w,
\end{aligned}
$$

其中 $\mathbf{A}^\mathrm{T} = [A_1^\mathrm{T}, A_2^\mathrm{T}, \ldots, A_N^\mathrm{T}]$，$D$ 是一个对角矩阵，对角线元素为 $D_{ii} = \sum_j S_{ij}$。$L = D - S$ 是拉普拉斯矩阵。矩阵 $D$ 为图的顶点提供了一个自然的度量，对应于原始图像。$D_{ii}$ 值越大（对应于第 $i$ 个图像），该顶点 $x_i$ 就越“重要”。此外，为了去除嵌入中的任意尺度因子，我们施加如下约束：

$$
\sum_i D_{ii} X_i^\mathrm{T} X_i = 1 \Rightarrow w^\mathrm{T} \left(\sum_i D_{ii} A_i^\mathrm{T} A_i\right) w = w^\mathrm{T} \mathbf{A}^\mathrm{T} (D \otimes I_m) \mathbf{A} w = 1.
$$


现在，最小化问题变为：
$$
\underset{w^\mathrm{T} \mathbf{A}^\mathrm{T} (D \otimes I_m) \mathbf{A} w = 1}{\operatorname*{\arg\min}} w^\mathrm{T} \mathbf{A}^\mathrm{T} (L \otimes I_m) \mathbf{A} w.
$$

最小化目标函数的变换向量 $w$ 由广义特征值问题的最小特征向量解给出：

$$
\mathbf{A}^\mathrm{T} (L \otimes I_m) \mathbf{A} w = \lambda \mathbf{A}^\mathrm{T} (D \otimes I_m) \mathbf{A} w.
$$

注意矩阵 $\mathbf{A}^\mathrm{T} (L \otimes I_m) \mathbf{A}$ 和 $\mathbf{A}^\mathrm{T} (D \otimes I_m) \mathbf{A}$ 都是对称且半正定的。最小化目标函数的向量 $w_i$ ($i = 1, 2, \ldots, d$) 是广义特征值问题的最小广义特征向量解。